<center>
<h1>Estimación de Área Glaciar utilizando Redes Neuronales Convolucionales U-Net en Imágenes Multiespectrales Sentinel 2 en el Glaciar Ausangate, 2019 </h1>
</center>

Este Codigo esta Inspirado en la presentacion de [Chris Brown & Nick Clinton EarthEngine + Tensorflow presentation,2018](https://www.youtube.com/watch?v=w-1xfF0IaeU). y el Repositorio GitHub publicado por [Cesar Aybar, 2019](https://github.com/csaybar/EEwPython/blob/master/cnn_demo.ipynb)

```
Modificado por :  Percy Elbis Colque Caillahua
Email : percyelbis@hotmail.com
Fecha : 28/07/2019
```

### 1. Instalar Librerias




In [0]:
!pip install -q tf-nightly-2.0-preview # tensorflow 2.0
!pip install earthengine-api==0.1.175 # earthengine API
!pip install -U -q PyDrive # Interact with Google drive
# Load the TensorBoard notebook extension
%load_ext tensorboard

### 2.  Importar Librerias.



In [0]:
# GCS and Google Drive.
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from oauth2client.client import GoogleCredentials

# Earth Engine Python API.
import ee 

import tensorflow as tf

# Makes it easy to visualize data.
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

### 3. Autenticacion.

In [0]:
# GCS and Google Drive.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# GEE.
!earthengine authenticate

In [0]:
ee.Initialize()

### 4. Funciones.

In [0]:
# Mapdisplay: Display ee.Features and ee.Images using folium.

def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = EE_TILES.format(**v),
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl()) # Lista de Layers.
    mapViz.save(outfile='map.html') # Save the map.
    return mapViz

In [0]:
# Funcion
'''
 * Function to mask clouds using the Sentinel-2 QA band.
 * Referencia : https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2.
 * Parametros : Image Sentinel-2 --->>> Nivel 2A.
 * Return : Devuelve los datos enmascarados y escalados, sin las bandas de control de calidad.
'''
def maskS2clouds(img):
  # Pixel QA band.
  qa = img.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = (1 << 10)
  cirrusBitMask = (1 << 11)
  # Get the pixel QA band.
  
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
           .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return img.updateMask(mask)\
            .divide(10000)\
            .copyProperties(img, ["system:time_start"])

### 5. Preparar Dataset

In [0]:
#---Datos de Entrenamiento/Validacion.
# Importar dataset de Entrenamiento y Validacion.
train_glaciar = ee.FeatureCollection('users/torchi_12/train_dataset_2019') 
test_glaciar = ee.FeatureCollection('users/torchi_12/test_dataset_2019')

# Display the train/test dataset
db_glx = train_glaciar.merge(test_glaciar)
center = db_glx.geometry().centroid().getInfo()['coordinates']
center.reverse()
#---Crear un Diccionario.
dicc = {'train': train_glaciar.draw(**{'color': 'FF0000', 'strokeWidth': 5}).getMapId(),
        'test' : test_glaciar.draw(**{'color': '0000FF', 'strokeWidth': 5}).getMapId(),
       }

#---Dataset Sentinel 2
'''
Sentinel-2 MSI: MultiSpectral Instrument, Level-2A
Dataset Availability : 2017-03-28T00:00:00 - Present
Referencia : https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
'''
# Selecionar Bandas.
bands = ['B12','B11','B8A','B8','B7','B6','B5','B4','B3','B2']
# Mar 28, 2017 - Jul 11, 2019 Se tine el Nivel 2A.
# En el area de entrenamiento solo se tiene desde 2018-12-13 hasta 2019-06-29.
s2_sr = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterBounds(db_glx)\
               .filterDate('2018-12-12', '2019-06-30')\
               .map(maskS2clouds)\
               .median()

# Load target.
target_glx = ee.Image('users/torchi_12/target_glx_s2').eq(1).rename('target')

# Rename Bands.
s2_msi = s2_sr.select(bands).rename(['SWIR2','SWIR1','RE4','NIR','RE3','RE2','RE1','R','G','B'])
# Add Bands s2_ndsi,target.
s2 = s2_msi.addBands(target_glx)
#---Preparar Dataset para Visualizacion.
from collections import OrderedDict
## Target.
target_glx_id = target_glx.getMapId()
dicc['target_glx'] = target_glx_id

# Crear Parametros de Visualizacion.
visParams_s2 = {    
  'bands': ['SWIR1', 'NIR', 'R'],
  'min': 0,
  'max': 0.5,
  'gamma': 1.4,
}

s2Mapid = s2.getMapId(visParams_s2)
dicc['Sentinel2'] = s2Mapid

# Change the dictionary order.
key_order = ['target_glx','Sentinel2','train','test']
dicc = OrderedDict((k, dicc[k]) for k in key_order)

# Show.
Mapdisplay(center,dicc,zoom_start=5)


In [0]:
import numpy as np
import time

def saveCNN_batch(image, point,kernel_size,scale,FilePrefix, selectors,folder, bucket='tesis_dataset'):
  """
    Export a dataset for semantic segmentation by batches
  
  Params:
  ------
    - image : ee.Image to get pixels from; must be scalar-valued.
    - point : Points to sample over.
    - kernel_size : The kernel specifying the shape of the neighborhood. Only fixed, square and rectangle kernels are supported.
      Weights are ignored; only the shape of the kernel is used.
    - scale : A nominal scale in meters of the projection to work in.
    - FilePrefix : Cloud Storage object name prefix for the export.
    - selector : Specified the properties to save.
    - bucket : The name of a Cloud Storage bucket for the export.  
  """
  print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + bucket) 
    else 'Output Cloud Storage bucket does not exist.')
  
  # Download the points (Server -> Client)
  nbands = len(selectors)
  points = train_glaciar.geometry().getInfo()['coordinates']    
  nfeatures = kernel_size*kernel_size*nbands*len(points) #estimate the totals # of features
     
  image_neighborhood = image.neighborhoodToArray(ee.Kernel.rectangle(kernel_size, kernel_size, 'pixels'))
  filenames = []
  
  #Threshold considering the max number of features permitted to export.
  if nfeatures > 3e6:
    nparts = int(np.ceil(nfeatures/3e6))
    print('Dataset too long, splitting it into '+ str(nparts),'equal parts.')
    
    nppoints = np.array(points)
    np.random.shuffle(nppoints)
    
    count_batch = 1  # Batch counter 
    
    for batch_arr in np.array_split(nppoints,nparts):
      
      fcp = ee.FeatureCollection([
          ee.Feature(ee.Geometry.Point(p),{'class':'NA'}) 
          for p in batch_arr.tolist() 
      ])
      
      # GLX dataset (fcp-points) collocation to each S2 grid cell value.
      train_db = image_neighborhood.sampleRegions(collection=fcp, scale=scale)
      filename = '%s/%s-%04d_' % (folder,FilePrefix,count_batch)
      
      # Create the tasks for passing of GEE to Google storage
      print('sending the task #%04d'%count_batch)
      Task = ee.batch.Export.table.toCloudStorage(
        collection=train_db,        
        selectors=selectors,          
        description='Export batch '+str(count_batch),
        fileNamePrefix=filename,
        bucket=bucket,  
        fileFormat='TFRecord')
      
      Task.start()
      filenames.append(filename)
      count_batch+=1
      
      while Task.active():
        print('Polling for task (id: {}).'.format(Task.id))
        time.sleep(3)
        
    return filenames
  
  else:    
    train_db = image_neighborhood.sampleRegions(collection=points, scale=scale)         
    Task = ee.batch.Export.table.toCloudStorage(
      collection=train_db,
      selectors=selectors,
      description='Training Export',
      fileNamePrefix=FilePrefix,
      bucket=bucket,  
      fileFormat='TFRecord')
    Task.start()
    
    while Task.active():
      print('Polling for task (id: {}).'.format(Task.id))
      time.sleep(3)
    
    return FilePrefix

In [0]:
selectors = ['SWIR2','SWIR1','RE4','NIR','RE3','RE2','RE1','R','G','B','target']
train_filenames = saveCNN_batch(s2,train_glaciar,128,10,'trainUNET', selectors,folder ='unet', bucket='tesis_dataset')
test_filenames = saveCNN_batch(s2,test_glaciar,128,10,'testUNET', selectors,folder ='unet', bucket='tesis_dataset')

### 6. Creating a tf.data.Dataset from a TFRecord file







In [0]:
# Fullname train/test db
folder = 'unet'
bucket = 'tesis_dataset'

filesList = !gsutil ls 'gs://'{bucket}'/'{folder}

trainFilePrefix = 'trainUNET'
trainFilePath = [s for s in filesList if trainFilePrefix in s]


testFilePrefix = 'testUNET'
testFilePath = [s for s in filesList if testFilePrefix in s]

In [0]:
def input_fn(fileNames, numEpochs=None, shuffle=True, batchSize=16, side = 257):
  # Read `TFRecordDatasets` 
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')

  # Names of the features 
  feature_columns = {
    'SWIR2' : tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'SWIR1' : tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'RE4': tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'NIR' : tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'RE3': tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'RE2': tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'RE1': tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'R': tf.io.FixedLenFeature([side,side], dtype=tf.float32),  
    'G': tf.io.FixedLenFeature([side,side], dtype=tf.float32),  
    'B': tf.io.FixedLenFeature([side,side], dtype=tf.float32),
    'target': tf.io.FixedLenFeature([side, side], dtype=tf.float32)
  }
  # Make a parsing function
  def parse(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, feature_columns)   
    # passing of 257x257 to 256x256
    parsed_features = {key:value[1:side,1:side] for key,value in parsed_features.items()} 
    # Separate the class labels from the training features
    labels = parsed_features.pop('target')
    return parsed_features, tf.cast(labels, tf.float32)
  
  # Passing of FeatureColumns to a 4D tensor
  def stack_images(features,label):         
    nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))
    nlabel = (tf.transpose(label))[:,:,tf.newaxis]
    return nfeat, nlabel
  
  dataset = dataset.map(parse, num_parallel_calls=4)
  dataset = dataset.map(stack_images, num_parallel_calls=4)
  
  if shuffle:
    dataset = dataset.shuffle(buffer_size = batchSize * 10)
  dataset = dataset.batch(batchSize)
  dataset = dataset.repeat(numEpochs)
  
  return dataset

In [0]:
train_dba = input_fn(trainFilePath,100,True,3)
test_dba = input_fn(testFilePath, numEpochs=1, batchSize=1, shuffle=False)

### 7. Visualizar


In [0]:
import matplotlib.pyplot as plt
import numpy as np

display_num = 5
plt.figure(figsize=(14, 21))

c=0
for i in range(1, display_num):
  for x in test_dba.take(i):
    x  
  tensor = tf.squeeze(x[0]).numpy()[:,:,[3,1,0]]
  target = tf.squeeze(x[1])

  # print(target.sum())  
  plt.subplot(display_num, 2, c + 1)
  plt.imshow(tensor)
  plt.title("RGB SENTINEL 2")
  
  plt.subplot(display_num, 2, c + 2)
  plt.imshow(target)
  plt.title("Glx Area")
  c+=2 
plt.show()

### 8. Preparar Entrada

In [0]:
# Definir la Forma de la Imagen de Entrada.
IMG_SHAPE  = (256, 256, 10)
EPOCHS = 20

### 9. Modelo U-Net


In [0]:
from tensorflow.keras import layers

def conv_block(input_tensor, num_filters):
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor) # Los filtros son 3x3--->Capa  de Convolucion
  encoder = layers.BatchNormalization()(encoder) # Capa de Normalizacion
  encoder = layers.Activation('relu')(encoder) # Capa de Activacion
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
  encoder = layers.BatchNormalization()(encoder)
  encoder = layers.Activation('relu')(encoder)
  return encoder

def encoder_block(input_tensor, num_filters):
  encoder = conv_block(input_tensor, num_filters)
  encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
  
  return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
  decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
  decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  return decoder

inputs = layers.Input(shape=IMG_SHAPE)
# 256
encoder0_pool, encoder0 = encoder_block(inputs, 32)
# 128
encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
# 64
encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
# 32
encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
# 16
encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
# 8
center = conv_block(encoder4_pool, 1024)
# center
decoder4 = decoder_block(center, encoder4, 512)
# 16
decoder3 = decoder_block(decoder4, encoder3, 256)
# 32
decoder2 = decoder_block(decoder3, encoder2, 128)
# 64
decoder1 = decoder_block(decoder2, encoder1, 64)
# 128
decoder0 = decoder_block(decoder1, encoder0, 32)
# 256

outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

### 10. Define your model


In [0]:
from tensorflow.keras import models
model = models.Model(inputs=[inputs], outputs=[outputs])
model.summary()

In [0]:
# Save the model.summary()
from contextlib import redirect_stdout

with open('modelsummary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

### 11. Defining custom metrics and loss functions

In [0]:
from tensorflow.keras import losses

def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
  loss = losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
  return loss

### 12. Compiling the model

In [0]:
# Compilamos el Modelo.
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_loss,'accuracy'])

In [0]:
from tensorflow.keras.utils import plot_model
plot_model(model)

### 13. Entrenar el Modelo

In [0]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import datetime

# Callbacks time
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
es = EarlyStopping(monitor='val_loss', patience=10)
mcp = ModelCheckpoint(filepath='UNet_Glx.h5', monitor='val_loss', save_best_only=True)

In [46]:
# Clear any logs from previous runs
!rm -rf ./logs/ 
# remove  carpeta
!rm UNet_Glx.h5 #rm fichero

rm: cannot remove 'UNet_Glx.h5': No such file or directory


In [0]:
#### Entrenamiento Opcional ya se tiene un modelo pre-entrenado guardado en google drive.####
N_train = 1883
# Validacion
N_test = 1470

# Tamaño de Lote.
batch_size = 10

# Train the model.
history = model.fit(train_dba,
                    validation_data=test_dba,
                    steps_per_epoch= int(np.ceil(N_train / float(batch_size))),
                    validation_steps= int(np.ceil(N_test / float(batch_size))),
                    epochs=EPOCHS,
                    callbacks=[tensorboard_callback,es,mcp])

In [0]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
%tensorboard --logdir logs/fit

### 14. Cargar el modelo pre-entrenado(20 epochs)


In [0]:
# Load the Model Pre-Entrenado of the GooGle Drive
file_obj =drive.CreateFile({'id':'1_1M9_zziF753YSVWEx3nKFZx0uJJtglk'}) # Desde Google Drive.
file_obj.GetContentFile('UNet_Glx.h5')

In [49]:
# Load el Modelo entrenado.

model.load_weights("UNet_Glx.h5")
model.evaluate(x = test_dba)

W0901 17:54:21.995866 140057857267584 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


1052/1052 [==============================] - 905s 861ms/step - loss: 0.0839 - dice_loss: 0.0286 - accuracy: 0.9788


[0.0839330828562733, 0.028609747, 0.9788186]

### 15. Prediction

In [0]:
# Area Glaciar a Predecir.
xmin,ymin,xmax,ymax = [-71.2895576800648882,-13.8284791303437160, -71.1695138486407046,-13.7602836898167542]

# Qgis Extension 
#-71.2895576800648882,-13.8284791303437160 : -71.1695138486407046,-13.7602836898167542---> Ausangate
#-70.8966675314120351,-13.9960341865535920 : -70.7458947766221655,-13.8373832645521784---> Quelccaya
#-77.6893128297523248,-9.1858434409387861 : -77.5221655263183607,-9.0342021137613369---> Huascaran
#-73.04705802581952,-50.4798111720673,-72.73120109222577,-50.310848619751894--->Perito
# Passing a rectangle (prediction area) to Earth Engine
ausangate = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])

In [0]:
# Desde Asset
'''
s2 = ee.Image('users/torchi_12/dat_s2/subset_6_of_S2A_MSIL2A_20190803T145731_N0213_R039_T19LBE_20190803T191158_resampled').multiply(0.0001)
sencor = ['b10','b9','b8','b7','b6','b5','b4','b3','b2','b1']

# renombrar bandas.
s2p = s2.select(sencor).rename(['SWIR2','SWIR1','RE4','NIR','RE3','RE2','RE1','R','G','B'])
#Fecha
formato = '20190803'
print(formato)
'''

In [52]:
# Desde GEE
s2 = ee.Image('COPERNICUS/S2_SR/20190818T145729_20190818T150545_T19LBE')
# Normalizando 0-1
s2_msi = s2.multiply(0.0001)
# Renombrar bandas.
s2p = s2_msi.select(bands).rename(['SWIR2','SWIR1','RE4','NIR','RE3','RE2','RE1','R','G','B'])
# Extraer la Fecha.
date = ee.Date(s2.get('system:time_start'))
fechasat= date.format('YMd').getInfo()
print(fechasat)

2019818


In [0]:
outputBucket = 'tesis_dataset'
imageFilePrefix = 'unet/Predict_Glx_ausangate2019819'

# Specify patch and file dimensions.
imageExportFormatOptions = {
  'patchDimensions': [256, 256],
  'compressed': True
}

# Setup the task.
imageTask = ee.batch.Export.image.toCloudStorage(
  image=s2p,
  description='Image Export',
  fileNamePrefix=imageFilePrefix,
  bucket=outputBucket,
  scale=10,
  fileFormat='TFRecord',
# Ampliamos el area de estudio a un buffer 3km.
  region = ausangate.buffer(3000).getInfo()['coordinates'],
  #region = ausangate.getInfo()['coordinates'],
  formatOptions=imageExportFormatOptions,
)

imageTask.start()

In [70]:
import time 
while imageTask.active():
  print('Polling for task (id: {}).'.format(imageTask.id))
  time.sleep(5)

Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).
Polling for task (id: ZY4OBZFIFFLYBBCLBYXHKYJS).


In [71]:
filesList = !gsutil ls 'gs://'{outputBucket}'/unet/'
exportFilesList = [s for s in filesList if imageFilePrefix in s]

# Get the list of image files and the JSON mixer file.
imageFilesList = []
jsonFile = None
for f in exportFilesList:
  if f.endswith('.tfrecord.gz'):
    imageFilesList.append(f)
  elif f.endswith('.json'):
    jsonFile = f

# Make sure the files are in the right order.
print(jsonFile)

gs://tesis_dataset/unet/Predict_Glx_ausangate2019819.json


In [72]:
import json
from pprint import pprint 
# Mixer contiene Metadat y Georeferenciacion
# Load the contents of the mixer file to a JSON object.
jsonText = !gsutil cat {jsonFile}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(jsonText.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 7,
 'projection': {'affine': {'doubleMatrix': [10.0,
                                            0.0,
                                            249420.0,
                                            0.0,
                                            -10.0,
                                            8480720.0]},
                'crs': 'EPSG:32719'},
 'totalPatches': 35}


In [0]:
def predict_input_fn(fileNames,side,bands):
  
  # Read `TFRecordDatasets` 
  dataset = tf.data.TFRecordDataset(fileNames, compression_type='GZIP')

  featuresDict = {x:tf.io.FixedLenFeature([side, side], dtype=tf.float32) for x in bands}
     
  # Make a parsing function
  def parse_image(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, featuresDict)
    return parsed_features
  
  def stack_images(features):         
    nfeat = tf.transpose(tf.squeeze(tf.stack(list(features.values()))))    
    return nfeat
  dataset = dataset.map(parse_image, num_parallel_calls=4)
  dataset = dataset.map(stack_images, num_parallel_calls=4)   
  dataset = dataset.batch(side*side)

  return dataset

In [0]:
predict_db = predict_input_fn(fileNames=imageFilesList,side=256,bands=['SWIR2','SWIR1','RE4','NIR','RE3','RE2','RE1','R','G','B'])
predictions = model.predict(predict_db)

In [75]:
# Instantiate the writer.
PATCH_WIDTH , PATCH_HEIGHT = [256,256]
outputImageFile = 'gs://' + outputBucket + '/unet/GlxAusangate.TFRecord'
writer = tf.io.TFRecordWriter(outputImageFile)

# Every patch-worth of predictions we'll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
curPatch = 1
for  prediction in predictions:
  patch = prediction.squeeze().T.flatten().tolist()
  
  if (len(patch) == PATCH_WIDTH * PATCH_HEIGHT):
    print('Done with patch ' + str(curPatch) + '...')    
    # Create an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'glx_prob': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch))
        }
      )
    )
    
    writer.write(example.SerializeToString())    
    curPatch += 1 

writer.close()

Done with patch 1...
Done with patch 2...
Done with patch 3...
Done with patch 4...
Done with patch 5...
Done with patch 6...
Done with patch 7...
Done with patch 8...
Done with patch 9...
Done with patch 10...
Done with patch 11...
Done with patch 12...
Done with patch 13...
Done with patch 14...
Done with patch 15...
Done with patch 16...
Done with patch 17...
Done with patch 18...
Done with patch 19...
Done with patch 20...
Done with patch 21...
Done with patch 22...
Done with patch 23...
Done with patch 24...
Done with patch 25...
Done with patch 26...
Done with patch 27...
Done with patch 28...
Done with patch 29...
Done with patch 30...
Done with patch 31...
Done with patch 32...
Done with patch 33...
Done with patch 34...
Done with patch 35...


In [76]:
## Upload the classifications to an EE asset.
!gsutil ls -l {outputImageFile}

   9176650  2019-09-01T18:36:43Z  gs://tesis_dataset/unet/GlxAusangate.TFRecord
TOTAL: 1 objects, 9176650 bytes (8.75 MiB)


In [77]:
# REPLACE WITH YOUR USERNAME:
USER_NAME = 'torchi_12'
outputAssetID = 'users/' + USER_NAME + '/GlxAusangate_' + fechasat
print('Writing to ' + outputAssetID)

Writing to users/torchi_12/GlxAusangate_2019818


###  16. Resultados GCS a GEE Asset

In [78]:
# Start the upload. It step might take a while.
!earthengine upload image --asset_id={outputAssetID} {outputImageFile} {jsonFile}

Started upload task with ID: EZABJANW33BNCSKTO3NBCXXP


In [85]:
## Display the Results.
ProbsImage = ee.Image(outputAssetID)
predictionsImage = ee.Image(outputAssetID).gte(0.500)
dicc = {'GlxProbability':ProbsImage.getMapId({'min':0.49,'max':0.498}),
        'Glx':predictionsImage.getMapId(),
       'S2_RGB': s2p.getMapId(visParams_s2)}

center = ausangate.centroid().getInfo()['coordinates']
center.reverse()

Mapdisplay(center=center,dicc=dicc,zoom_start=13)